In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('722-model').getOrCreate()

In [2]:
data1= spark.read.csv("./data_japan.csv", header=True, inferSchema=True)
data1.printSchema()

root
 |-- adm0_id: integer (nullable = true)
 |-- adm0_name: string (nullable = true)
 |-- adm1_id: integer (nullable = true)
 |-- adm1_name: string (nullable = true)
 |-- mkt_id: integer (nullable = true)
 |-- mkt_name: string (nullable = true)
 |-- cm_id: integer (nullable = true)
 |-- cm_name: string (nullable = true)
 |-- cur_id: integer (nullable = true)
 |-- cur_name: string (nullable = true)
 |-- pt_id: integer (nullable = true)
 |-- pt_name: string (nullable = true)
 |-- um_id: integer (nullable = true)
 |-- um_name: string (nullable = true)
 |-- mp_month: integer (nullable = true)
 |-- mp_year: integer (nullable = true)
 |-- mp_price_RMB: double (nullable = true)



In [3]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
  inputCols=['mp_price_RMB'],
              outputCol="features")
output = assembler.transform(data1)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="cm_name", outputCol="PrivateIndex")
output_fixed = indexer.fit(output).transform(output)
final_data = output_fixed.select("features",'PrivateIndex')

In [4]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline
dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
train_data,test_data = final_data.randomSplit([0.7,0.3])
dtc_model = dtc.fit(train_data)
dtc_predictions = dtc_model.transform(test_data)

In [5]:
# Let's start off with binary classification.
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Note that the label column isn't named label, it's named PrivateIndex in this case.
my_binary_eval = BinaryClassificationEvaluator(labelCol = 'PrivateIndex')

In [6]:
print("DTC")
print(my_binary_eval.evaluate(dtc_predictions))

DTC
0.5649700598802396


In [7]:
dtc_predictions.show()
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="accuracy")
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
print('-'*40)
print('A single decision tree has an accuracy of: {0:2.2f}%'.format(dtc_acc*100))


+-------------------+------------+--------------------+--------------------+----------+
|           features|PrivateIndex|       rawPrediction|         probability|prediction|
+-------------------+------------+--------------------+--------------------+----------+
|[7.823999999999999]|         3.0|[0.0,0.0,3.0,65.0...|[0.0,0.0,0.027272...|       3.0|
|[8.149999999999999]|         6.0|[0.0,0.0,3.0,65.0...|[0.0,0.0,0.027272...|       3.0|
|[8.475999999999999]|         3.0|[0.0,0.0,3.0,65.0...|[0.0,0.0,0.027272...|       3.0|
|           [8.6716]|         6.0|[0.0,0.0,3.0,65.0...|[0.0,0.0,0.027272...|       3.0|
|           [8.9324]|         3.0|[0.0,0.0,3.0,65.0...|[0.0,0.0,0.027272...|       3.0|
|[8.997599999999998]|         3.0|[0.0,0.0,3.0,65.0...|[0.0,0.0,0.027272...|       3.0|
|           [9.0628]|         6.0|[0.0,0.0,3.0,65.0...|[0.0,0.0,0.027272...|       3.0|
|[9.323599999999999]|         6.0|[0.0,0.0,3.0,65.0...|[0.0,0.0,0.027272...|       3.0|
|[9.453999999999999]|         6.

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
final_data.show()
data1.select('cm_name').distinct().show()
final_data.select('PrivateIndex').distinct().show()

+--------------------+------------+
|            features|PrivateIndex|
+--------------------+------------+
|          [139.4628]|         1.0|
|           [136.268]|         1.0|
|          [137.9632]|         1.0|
|[134.70319999999998]|         1.0|
|[136.85479999999998]|         1.0|
|[134.70319999999998]|         1.0|
|          [147.2216]|         1.0|
|[146.11319999999998]|         1.0|
|[149.89479999999998]|         1.0|
|          [148.7864]|         1.0|
|           [147.678]|         1.0|
|[147.80839999999998]|         1.0|
|[149.95999999999998]|         1.0|
|          [150.4816]|         1.0|
|          [153.0896]|         1.0|
|          [154.7848]|         1.0|
|[155.82799999999997]|         1.0|
|[155.37159999999997]|         1.0|
|          [159.5444]|         1.0|
|[160.13119999999998]|         1.0|
+--------------------+------------+
only showing top 20 rows

+--------------------+
|             cm_name|
+--------------------+
|Wheat flour - Retail|
|      Sugar - Ret

In [15]:
print(dtc_model.toDebugString)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4d93a6c17cb9d39467bc) of depth 5 with 37 nodes
  If (feature 0 <= 5.9136)
   If (feature 0 <= 3.0864)
    If (feature 0 <= 2.7312)
     If (feature 0 <= 1.3374)
      Predict: 1.0
     Else (feature 0 > 1.3374)
      If (feature 0 <= 1.8696)
       Predict: 2.0
      Else (feature 0 > 1.8696)
       Predict: 1.0
    Else (feature 0 > 2.7312)
     If (feature 0 <= 2.891904)
      Predict: 4.0
     Else (feature 0 > 2.891904)
      Predict: 2.0
   Else (feature 0 > 3.0864)
    If (feature 0 <= 4.8)
     If (feature 0 <= 4.08)
      If (feature 0 <= 3.39374424)
       Predict: 4.0
      Else (feature 0 > 3.39374424)
       Predict: 4.0
     Else (feature 0 > 4.08)
      Predict: 4.0
    Else (feature 0 > 4.8)
     Predict: 4.0
  Else (feature 0 > 5.9136)
   If (feature 0 <= 15.5784)
    If (feature 0 <= 6.84)
     Predict: 5.0
    Else (feature 0 > 6.84)
     If (feature 0 <= 12.336)
      If (feature 0 <= 11.2008)
       Predict